## Importing Libraries

In [121]:
import json
import pickle
import random

import nltk
import nltk.data
from nltk.stem import WordNetLemmatizer

import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.layers import Dense, Activation, Dropout

In [122]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/rebeccaxie/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [123]:
with open(r'train_light.json') as data:
    contents = json.loads(data.read())

In [124]:
questions = []
question_types = []
question_and_question_types = []


for content in contents:
    question = nltk.word_tokenize(content['question'])
    questions.extend(question)
    question_types.extend(content['annotations'][0]['type'].split())
    question_and_question_types.append((question, content['annotations'][0]['type']))
    
print(question_and_question_types)


[(['When', 'did', 'the', 'simpsons', 'first', 'air', 'on', 'television', '?'], 'multipleQAs'), (['Who', 'played', 'george', 'washington', 'in', 'the', 'john', 'adams', 'series', '?'], 'singleAnswer'), (['What', 'is', 'the', 'legal', 'age', 'of', 'marriage', 'in', 'usa', '?'], 'multipleQAs'), (['Who', 'starred', 'in', 'barefoot', 'in', 'the', 'park', 'on', 'broadway', '?'], 'multipleQAs'), (['When', 'did', 'the', 'manhattan', 'project', 'began', 'and', 'end', '?'], 'multipleQAs'), (['When', 'did', 'the', 'frozen', 'ride', 'open', 'at', 'epcot', '?'], 'singleAnswer'), (['Name', 'the', 'landforms', 'that', 'form', 'the', 'boundaries', 'of', 'the', 'peninsular', 'plateau', '?'], 'singleAnswer'), (['When', 'was', 'the', 'last', 'time', 'uga', 'won', 'a', 'national', 'championship', '?'], 'multipleQAs'), (['Who', 'sing', 'play', 'that', 'funky', 'music', 'white', 'boy', '?'], 'multipleQAs'), (['When', 'was', 'the', 'first', 'airplane', 'used', 'in', 'war', '?'], 'singleAnswer'), (['What', 'c

In [125]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/rebeccaxie/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [126]:
lemmatizer = WordNetLemmatizer()
to_remove = ['?']

questions = [lemmatizer.lemmatize(question.lower()) for question in questions if question not in to_remove]
questions = sorted(list(set(questions)))
question_types = sorted(list(set(question_types)))

pickle.dump(questions, open('questions.pkl', 'wb'))
pickle.dump(question_types, open('question_types.pkl', 'wb'))

In [127]:
training = []
output_initialization = [0] * len(question_types)

for question_and_question_type in question_and_question_types:
    bag = []
    questions_part = question_and_question_type[0]
    questions_part = [lemmatizer.lemmatize(q.lower()) for q in questions_part]
    for question in questions:
        #print(question)
        if question in questions_part:
            bag.append(1)
        else:
            bag.append(0)
    row = list(output_initialization)
    row[question_types.index(question_and_question_types[1][1])] = 1
    training.append([bag, row]) 

In [128]:
random.shuffle(training)
training = np.array(training)
X_train = list(training[:, 0])
y_train = list(training[:, 1])

<ipython-input-128-057d101b40f1>:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


In [129]:
model = Sequential()
model.add(Dense(128, input_shape=(len(X_train[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(y_train[0]), activation='softmax'))

In [130]:
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

hist = model.fit(np.array(X_train),
                 np.array(y_train),
                 epochs=200,
                 batch_size=5,
                 verbose=1)

model.save('model', hist)

Epoch 1/200
2008/2008 [==============================] - 8s 4ms/step - loss: 0.0059 - accuracy: 0.9995
Epoch 2/200
2008/2008 [==============================] - 8s 4ms/step - loss: 1.8480e-04 - accuracy: 1.0000
Epoch 3/200
2008/2008 [==============================] - 8s 4ms/step - loss: 1.0780e-04 - accuracy: 1.0000
Epoch 4/200
2008/2008 [==============================] - 9s 4ms/step - loss: 8.0625e-05 - accuracy: 1.0000
Epoch 5/200
2008/2008 [==============================] - 8s 4ms/step - loss: 5.7777e-05 - accuracy: 1.0000
Epoch 6/200
2008/2008 [==============================] - 8s 4ms/step - loss: 4.3935e-05 - accuracy: 1.0000
Epoch 7/200
2008/2008 [==============================] - 9s 4ms/step - loss: 4.1445e-05 - accuracy: 1.0000
Epoch 8/200
2008/2008 [==============================] - 9s 4ms/step - loss: 2.8570e-05 - accuracy: 1.0000
Epoch 9/200
2008/2008 [==============================] - 8s 4ms/step - loss: 2.6571e-05 - accuracy: 1.0000
Epoch 10/200
2008/2008 [=================

Epoch 77/200
2008/2008 [==============================] - 8s 4ms/step - loss: 2.7941e-06 - accuracy: 1.0000
Epoch 78/200
2008/2008 [==============================] - 8s 4ms/step - loss: 4.3258e-06 - accuracy: 1.0000
Epoch 79/200
2008/2008 [==============================] - 8s 4ms/step - loss: 2.7511e-06 - accuracy: 1.0000
Epoch 80/200
2008/2008 [==============================] - 9s 4ms/step - loss: 2.7572e-06 - accuracy: 1.0000
Epoch 81/200
2008/2008 [==============================] - 8s 4ms/step - loss: 2.5772e-06 - accuracy: 1.0000
Epoch 82/200
2008/2008 [==============================] - 8s 4ms/step - loss: 1.8353e-06 - accuracy: 1.0000
Epoch 83/200
2008/2008 [==============================] - 8s 4ms/step - loss: 2.3241e-06 - accuracy: 1.0000
Epoch 84/200
2008/2008 [==============================] - 9s 4ms/step - loss: 2.6173e-06 - accuracy: 1.0000
Epoch 85/200
2008/2008 [==============================] - 8s 4ms/step - loss: 3.2372e-06 - accuracy: 1.0000
Epoch 86/200
2008/2008 [====

2008/2008 [==============================] - 8s 4ms/step - loss: 3.9915e-06 - accuracy: 1.0000
Epoch 153/200
2008/2008 [==============================] - 8s 4ms/step - loss: 1.1789e-06 - accuracy: 1.0000
Epoch 154/200
2008/2008 [==============================] - 8s 4ms/step - loss: 1.4009e-06 - accuracy: 1.0000
Epoch 155/200
2008/2008 [==============================] - 8s 4ms/step - loss: 2.2085e-06 - accuracy: 1.0000
Epoch 156/200
2008/2008 [==============================] - 8s 4ms/step - loss: 1.7139e-06 - accuracy: 1.0000
Epoch 157/200
2008/2008 [==============================] - 9s 4ms/step - loss: 1.4805e-06 - accuracy: 1.0000
Epoch 158/200
2008/2008 [==============================] - 8s 4ms/step - loss: 1.3891e-06 - accuracy: 1.0000
Epoch 159/200
2008/2008 [==============================] - 8s 4ms/step - loss: 1.2591e-06 - accuracy: 1.0000
Epoch 160/200
2008/2008 [==============================] - 8s 4ms/step - loss: 2.1555e-06 - accuracy: 1.0000
Epoch 161/200
2008/2008 [========